In [1]:
# Create data directory if it doesn't exist
!mkdir -p ./data

# Download patent files to data directory
!wget -P ./data https://s3.amazonaws.com/data.patentsview.org/download/g_patent.tsv.zip
!wget -P ./data https://s3.amazonaws.com/data.patentsview.org/download/g_patent_abstract.tsv.zip

# Decompress the downloaded files
!unzip -o patents/data/g_patent.tsv.zip -d ./data
!unzip -o patents/data/g_patent_abstract.tsv.zip -d ./data

--2025-03-23 03:56:54--  https://s3.amazonaws.com/data.patentsview.org/download/g_patent.tsv.zip
Resolving s3.amazonaws.com (s3.amazonaws.com)... 16.182.96.144, 52.217.199.0, 52.216.146.69, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|16.182.96.144|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 223192856 (213M) [binary/octet-stream]
Saving to: ‘./data/g_patent.tsv.zip’

g_patent.tsv.zip    100%[===================>] 212.85M  27.6MB/s    in 7.9s    

2025-03-23 03:57:02 (27.0 MB/s) - ‘./data/g_patent.tsv.zip’ saved [223192856/223192856]

--2025-03-23 03:57:03--  https://s3.amazonaws.com/data.patentsview.org/download/g_patent_abstract.tsv.zip
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.206.117, 16.15.217.193, 16.15.217.123, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.206.117|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1647990737 (1.5G) [binary/octet-stream]
Saving to: ‘./data/g_patent_abstract.

In [2]:
import pandas as pd

# Load the data into pandas dataframes
patents = pd.read_csv('patents/data/g_patent.tsv', sep='\t')
abstracts = pd.read_csv('patents/data/g_patent_abstract.tsv', sep='\t')

/tmp/ipykernel_1552863/1218841530.py:4: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  patents = pd.read_csv('patents/data/g_patent.tsv', sep='\t')
/tmp/ipykernel_1552863/1218841530.py:5: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  abstracts = pd.read_csv('patents/data/g_patent_abstract.tsv', sep='\t')


In [3]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("allenai/specter2_base", use_fast=True, trust_remote_code=True)

/mnt/erzhuo-storage-us/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
patents['patent_abstract'] = abstracts.patent_abstract
patents['content'] = patents.patent_title + tokenizer.sep_token + patents.patent_abstract
patents.patent_id = patents.patent_id.astype(str)
patents = patents[patents.patent_abstract.notna()]

In [7]:
save_path = '/mnt/local/ii/retriever/patents'

patents.sample(frac=0.1).to_parquet(f'{save_path}/patents_sample.parquet')
patents.to_parquet(f'{save_path}/patents_all.parquet')

In [30]:
import datasets
datasets.load_dataset(
    'parquet',
    data_files='./data/patents.parquet',
    split='train',
    num_proc=4
)

Dataset({
    features: ['patent_id', 'patent_type', 'patent_date', 'patent_title', 'wipo_kind', 'num_claims', 'withdrawn', 'filename', 'patent_abstract', 'content'],
    num_rows: 10000
})